In [ ]:
# processes sea level rise data

In [1]:
import os
import sys
import math
import warnings
import yaml
import pandas as pd
import geopandas as gpd
from osgeo import gdal
import glob
import numpy as np
import rasterio.mask
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.merge import merge
from os.path import exists
from pathlib import Path
from shutil import copyfile

In [2]:
country = 'Albania'

In [4]:
aoi_folder = Path(r'C:\Users\Owner\Documents\Career\World Bank\CRP\Albania\data\AOI')
output_folder = Path(r'C:\Users\Owner\Documents\Career\World Bank\CRP\Albania\data')

# Raw data folder. Change file path as needed
data_folder = Path(r'D:\World Bank\CRP\data\Sea level rise\WB_Mar2023')

# create a corresponding folder on an external hard drive to store large raster files (intermediate outputs). Change file path as needed
int_output_folder = Path(r'D:\World Bank\CRP') / country / 'data/SLR'
cities = ['Shkoder', 'Vlore']
# centroids = pd.read_csv('centroids.csv')
epsg_dict = {'Shkoder': 32634, 'Vlore': 32634}
year_list = [2020, 2050, 2100]
slr_list = ['', '_RL10']
ssp_list = [245, 585]

In [5]:
try:
    os.mkdir(int_output_folder)
except FileExistsError:
    pass

In [6]:
# get country shapefile extent
shp_bounds = gpd.read_file(Path(r'C:\Users\Owner\Documents\Career\World Bank\CRP\Albania\data') / (country.replace(' ', '_') + '.shp')).bounds

Because the SLR data is organized by tiles of 1x1 degree, we first identify the tiles that cover the country extent so that they can be merged into one raster

In [7]:
def smart_append(element, ls):
    if not element in ls:
        ls.append(element)

In [8]:
lat_list = []

for i in range(len(shp_bounds)):
    if math.floor(shp_bounds.miny[i]) >= 0:
        hemi = 'N'
        for y in range(math.floor(shp_bounds.miny[i]), math.ceil(shp_bounds.maxy[i])):
            smart_append(hemi + str(y).zfill(2), lat_list)
    elif math.ceil(shp_bounds.maxy[i]) >= 0:
        for y in range(0, math.ceil(shp_bounds.maxy[i])):
            smart_append('N' + str(y).zfill(2), lat_list)
        for y in range(math.floor(shp_bounds.miny[i]), 0):
            smart_append('S' + str(-y).zfill(2), lat_list)
    else:
        hemi = 'S'
        for y in range(math.floor(shp_bounds.miny[i]), math.ceil(shp_bounds.maxy[i])):
            smart_append(hemi + str(-y).zfill(2), lat_list)

In [9]:
lon_list = []

for i in range(len(shp_bounds)):
    if math.floor(shp_bounds.minx[i]) >= 0:
        hemi = 'E'
        for x in range(math.floor(shp_bounds.minx[i]), math.ceil(shp_bounds.maxx[i])):
            smart_append(hemi + str(x).zfill(3), lon_list)
    elif math.ceil(shp_bounds.maxx[i]) >= 0:
        for x in range(0, math.ceil(shp_bounds.maxx[i])):
            smart_append('E' + str(x).zfill(3), lon_list)
        for x in range(math.floor(shp_bounds.minx[i]), 0):
            smart_append('W' + str(-x).zfill(3), lon_list)
    else:
        hemi = 'W'
        for x in range(math.floor(shp_bounds.minx[i]), math.ceil(shp_bounds.maxx[i])):
            smart_append(hemi + str(-x).zfill(3), lon_list)

In [10]:
# copy all the identified tiles into one folder
for year in year_list:
    for slr in slr_list:
        for ssp in ssp_list:
            try:
                os.mkdir(int_output_folder / ('ssp' + str(ssp) + '_mediumconfidence_50.0_' + str(year) + slr))
            except FileExistsError:
                pass
            for lat in lat_list:
                for lon in lon_list:
                    data_subfolder = 'AR6_ssp' + str(ssp) + '_mediumconfidence_50.0_' + str(year) + slr
                    if not exists(int_output_folder / ('ssp' + str(ssp) + '_mediumconfidence_50.0_' + str(year) + slr) / (lat + lon + '.tif')):
                        try:
                            copyfile(data_folder / data_subfolder / (lat + lon + '.tif'),
                                     int_output_folder / ('ssp' + str(ssp) + '_mediumconfidence_50.0_' + str(year) + slr) / (lat + lon + '.tif'))
                        except FileNotFoundError:
                            pass

In [11]:
# merge the tiles into one raster file
for year in year_list:
    for slr in slr_list:
        for ssp in ssp_list:
            raster_to_mosaic = []
            mosaic_file = 'ssp' + str(ssp) + '_mediumconfidence_50.0_' + str(year) + slr + '.tif'
            
            mosaic_list = list((int_output_folder / ('ssp' + str(ssp) + '_mediumconfidence_50.0_' + str(year) + slr)).iterdir())
            for p in mosaic_list:
                raster = rasterio.open(p)
                raster_to_mosaic.append(raster)
            
            mosaic, output = merge(raster_to_mosaic)
            output_meta = raster.meta.copy()
            output_meta.update(
                {"driver": "GTiff",
                    "height": mosaic.shape[1],
                    "width": mosaic.shape[2],
                    "transform": output,
                }
            )
            
            with rasterio.open(int_output_folder / mosaic_file, 'w', **output_meta) as m:
                m.write(mosaic)

In [14]:
# reproject the raster files as needed and clip them to the city extents
def clipdata_slr(slr, year, ssp, city):
    city_no_space = city.replace(" ", "_")
    city_lower = city_no_space.lower()
    crs = epsg_dict[city]
    shp_name = city_no_space + '.shp'
    shp = gpd.read_file(city / aoi_folder / shp_name).to_crs(epsg = crs)
    features = shp.geometry
    
    projected_raster = 'ssp' + str(ssp) + '_mediumconfidence_50.0_' + str(year) + slr + '_' + str(crs) + '.tif'
    unprojected_raster = 'ssp' + str(ssp) + '_mediumconfidence_50.0_' + str(year) + slr + '.tif'
    if not exists(int_output_folder / projected_raster):
        with rasterio.open(int_output_folder / unprojected_raster) as src:
            dst_crs = 'EPSG:' + str(crs)

            transform, width, height = calculate_default_transform(
                src.crs, dst_crs, src.width, src.height, *src.bounds)
            kwargs = src.meta.copy()
            kwargs.update({
                'crs': dst_crs,
                'transform': transform,
                'width': width,
                'height': height
            })

            with rasterio.open(int_output_folder / projected_raster, 'w', **kwargs) as dst:
                for i in range(1, src.count + 1):
                    reproject(
                        source=rasterio.band(src, i),
                        destination=rasterio.band(dst, i),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=dst_crs,
                        resampling=Resampling.nearest)
    
    try:
        with rasterio.open(int_output_folder / projected_raster) as src:
            out_image, out_transform = rasterio.mask.mask(
                src, features, crop=True)
            out_meta = src.meta.copy()

        out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})
        
        if np.nansum(out_image) != 0:
            out_file = city_lower + '_ssp' + str(ssp) + '_slr' + slr + "_" + str(year) + '.tif'
            with rasterio.open(city / output_folder / out_file, "w", **out_meta) as dest:
                dest.write(out_image)
    except ValueError:
        pass

In [15]:
for city in cities:
    for slr in slr_list:
        for year in year_list:
            for ssp in ssp_list:
                clipdata_slr(slr, year, ssp, city)